In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/animeplanet-recommendation-database-2020/watching_status.csv
/kaggle/input/animeplanet-recommendation-database-2020/rating_complete.csv
/kaggle/input/animeplanet-recommendation-database-2020/anime_recommendations.csv
/kaggle/input/animeplanet-recommendation-database-2020/animelist.csv
/kaggle/input/animeplanet-recommendation-database-2020/anime.csv
/kaggle/input/animeplanet-recommendation-database-2020/html folder/instructions.txt
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/reviews_1.html
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/characters.html
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/staff.html
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/recommendations_3.html
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/recommendations_2.html
/kaggle/input/animeplanet-recommendation-database-2020/html folder/html/2/recommendations_1.h

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
file = pd.read_csv('../input/animeplanet-recommendation-database-2020/anime.csv')
file = file.reset_index()
file.head()

,index,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


In [6]:
file.columns

Index(['index', 'Anime-PlanetID', 'Name', 'Alternative Name', 'Rating Score',
       'Number Votes', 'Tags', 'Content Warning', 'Type', 'Episodes',
       'Finished', 'Duration', 'StartYear', 'EndYear', 'Season', 'Studios',
       'Synopsis', 'Url'],
      dtype='object')

In [7]:
features = ['Rating Score','Number Votes','Studios','Synopsis', 'Tags', 'Episodes']

In [8]:
def combined_features(row):
    return str(row["Rating Score"])+" "+ str(row["Number Votes"])+" "+ str(row["Studios"])+" "+ str(row["Synopsis"])+" "+ str(row["Tags"])+" "+ str(row["Episodes"])+" "

def get_title_from_index(index):
    return file[file["index"] == index]["Name"].values[0]
def get_index_from_title(title):
    return file[file["Name"] == title]["index"].values[0]

In [9]:
file["combined_feature"]=file.apply(combined_features,axis=1)
file["combined_feature"].head()

0    4.037 10889 Production I.G, Trans Arts Meet Ry...
1    4.248 54463 GAINAX, Tatsunoko Production In th...
2    4.35 23948 Kyoto Animation Half a year has pas...
3    2.8 131 Unknown The idol group 22/7 perform th...
4    1.271 21 Toei Animation No synopsis yet - chec...
Name: combined_feature, dtype: object

In [10]:
cv = CountVectorizer()
count_matrix=cv.fit_transform(file["combined_feature"])

In [26]:
movie_liked_by_user = "Death Note"

In [20]:
cosine_sim = cosine_similarity(count_matrix)


In [27]:
liked_movie_index = cosine_sim[get_index_from_title(movie_liked_by_user)]
similar_anime = list(enumerate(liked_movie_index))
# similar_anime_sorted = sorted(similar_anime)
similar_anime.sort(key = lambda row: row[1],reverse=True)
# print(similar_anime)

for i in range(5):
    print(get_title_from_index(similar_anime[i][0]))

Death Note
Death Note Rewrite 1: Visions of a God
Rokka: Braves of the Six Flowers
Da Zhu Zai
Sword Art Online: Alicization - War of Underworld: Part II


In [14]:
# print(file['Name'].where((file['Anime-PlanetID']) == 10)).values[0]
# file[file["Anime-PlanetID"] == 10]["Name"]
# file[file["Name"] == "The Prince of Tennis"]["Anime-PlanetID"].values[0]
# get_index_from_title(movie_liked_by_user)
# get_title_from_index(0)
# cosine_sim
# file = file.reset_index()
# file.head()
import pickle
pickle.dump(file.to_dict(),open('./animes_dict.pkl','wb'))
similarity=cosine_sim
pickle.dump(similarity,open('./similarity.pkl','wb'))

